# Fortnight 2 Demo


In [1]:
from transformers import pipelin

summarizer = pipeline("summarization", model="google/pegasus-multi_news")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-multi_news and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from datasets import load_dataset

cc_news_data = load_dataset("cc_news" , split="train")

In [17]:
print(cc_news_data)

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 708241
    })
})


In [4]:
cc_news_data[10]


{'title': 'Isabella Boylston and James Whiteside Get Hilariously Candid',
 'text': "Though American Ballet Theatre principals James Whiteside and Isabella Boylston have long displayed their envy-worthy friendship on Instagram, this week the Cindies (their nickname for each other) offered viewers an even deeper glimpse into their world. While on tour with ABT at the Kennedy Center, the duo sat down in front of the camera to answer some questions from their fans via Facebook Live.\nStarbucks in hand, they discuss their mutual love of food (particularly pasta and Japanese curry), the story behind the Cindy nickname and what it's like picking up contemporary choreography versus classical. Boylston also delves into her experience guesting with the Paris Opéra Ballet, her dream of choreographing an avant-garde ballet on Whiteside to a Carly Rae Jepsen song and best and worst Kennedy Center memories (like the time she fell onstage while doing fouettés at the end of La Bayadère's first act).\n

In [5]:
len(cc_news_data[10]['text'])

1204

In [6]:
max_length = 0
max_index = 0
for i in range(len(cc_news_data)):
    if len(cc_news_data[i]['text']) > max_length:
        max_length = len(cc_news_data[i]['text'])
        max_index = i
        print(f"max_length: {max_length}, max_index: {max_index}")

max_length: 895, max_index: 0
max_length: 1215, max_index: 1
max_length: 4836, max_index: 4
max_length: 6473, max_index: 7
max_length: 7738, max_index: 8
max_length: 13455, max_index: 33
max_length: 14099, max_index: 37
max_length: 33930, max_index: 385
max_length: 40424, max_index: 5137
max_length: 46308, max_index: 7215
max_length: 77891, max_index: 14115
max_length: 86728, max_index: 28470
max_length: 96787, max_index: 54916
max_length: 99842, max_index: 161689
max_length: 99954, max_index: 237533
max_length: 99980, max_index: 237549
max_length: 100000, max_index: 489689


In [7]:
print(max_index)

489689


In [15]:
cc_news_data[14]

{'title': 'The Joffrey Presents Ekman\'s "Midsummer Night\'s Dream"',
 'text': 'This spring, The Joffrey Ballet will present the North American premiere of Alexander Ekman\'s Midsummer Night\'s Dream. The Swedish choreographer is best known for his absurdist and cutting-edge productions. "This is not Shakespeare\'s Midsummer," says Joffrey Ballet artistic director Ashley Wheater. The title of Ekman\'s version, which premiered with the Royal Swedish Ballet in 2015, refers not to Shakespeare but to Midsummer, the traditional Scandinavian summer solstice festival. The piece follows a young man through a day of revelry followed by a nightmare, blurring the line with reality. "It\'s a kind of otherworldly dream," says Wheater.\nBringing Ekman\'s production to life is no small feat; the piece utilizes the entire Joffrey company. "I can\'t think of another performance that has so many props," says Wheater, listing giant bales of hay, long banquet tables, umbrellas, beach chairs and more. The 

In [9]:
print(summarizer(cc_news_data[14115]['text'][:3975], max_length=100, min_length=50))

[{'summary_text': '– If you\'re looking for a fun way to spend your Saturday night, head to one of the UK\'s many comedy clubs. The Empire Theatre in Blackburn, for example, is hosting Grimethorpe Band, a "feel-good tribute performing the much-loved hits of the 70s legends," per the Lancashire Evening Post. Or you can head to Manchester\'s Manford\'s Comedy Club, where "Jason selects some of his favorite circuit comedians and sends them to Chorley'}]


In [10]:
list_indices_to_keep = []
for i in range(len(cc_news_data)):
    if len(cc_news_data[i]['text']) < 3000 and len(cc_news_data[i]['text']) > 1000:
        if len(cc_news_data[i]['description']) > 20:
            list_indices_to_keep.append(i)

In [11]:
print(len(list_indices_to_keep))

216884


In [12]:
# create a new dataset with the indices we want to keep
cc_news_data_short = cc_news_data.select(list_indices_to_keep)
cc_news_data_short

Dataset({
    features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
    num_rows: 216884
})

In [13]:
# get 5 random samples from the dataset
to_test = cc_news_data_short.shuffle().select(range(3))


In [14]:
# get 5 random samples from the dataset
to_test = cc_news_data_short.shuffle().select(range(3))


In [97]:
for each in to_test:
    sum = summarizer(each['text'], max_length=100, min_length=50)
    print(f"Actual description: {each['description']}\nPredicted summary: {sum}\n\n")

Actual description: Bradley, Red Sox complete sweep, Nationals fall below .500
Predicted summary: [{'summary_text': "– Struggling outfielder Jackie Bradley Jr. drove in two runs, Eduardo Rodriguez struck out six in six shutout innings, and the Boston Red Sox shut out the Washington Nationals 3-0 on Wednesday to complete a three-game sweep and send the preseason NL East favorites below .500. The Nationals (42-43) have lost five in a row to fall below .500 this late in the season for the first time since being 60-61 on Aug. 21, 2015. A throwing error by Adam Eaton on Bradley's"}]


Actual description: At Holland and Barrett they are now giving Free UK Delivery on all products sold on the hollandandbarrett.com website. Click to benefit from the Free UK
Predicted summary: [{'summary_text': '– If you\'re in the market for a new set of wheels, you could do a lot worse than pick up a set of wheels from Holland and Barrett. The health supplement retailer is currently offering half-price delive

In [98]:
references = []
predictions = []
for each in to_test:
    references.append(each['description'])
    predictions.append(summarizer(each['text'], max_length=100, min_length=50)[0]['summary_text'])
    

In [99]:
references

['Bradley, Red Sox complete sweep, Nationals fall below .500',
 'At Holland and Barrett they are now giving Free UK Delivery on all products sold on the hollandandbarrett.com website. Click to benefit from the Free UK',
 'AbRam is having a whale of a time with his fam jam and the pictures are a proofGet latest Bollywood online at in.com']

In [100]:
predictions

["– Struggling outfielder Jackie Bradley Jr. drove in two runs, Eduardo Rodriguez struck out six in six shutout innings, and the Boston Red Sox shut out the Washington Nationals 3-0 on Wednesday to complete a three-game sweep and send the preseason NL East favorites below .500. The Nationals (42-43) have lost five in a row to fall below .500 this late in the season for the first time since being 60-61 on Aug. 21, 2015. A throwing error by Adam Eaton on Bradley's",
 '– If you\'re in the market for a new set of wheels, you could do a lot worse than pick up a set of wheels from Holland and Barrett. The health supplement retailer is currently offering half-price delivery on all its products in the UK, according to a post at bargainbarrett.com. Shoppers can also take advantage of the company\'s "Get One Get One Free" offer.',
 "– Shah Rukh Khan's younger son, AbRam, is having a time of his life along with his parents and siblings— Aryan and Suhana—in Europe. Thanks to social media, we know 

In [101]:
import evaluate

rouge = evaluate.load("rouge")

rouge.compute(predictions=predictions, references=references)

{'rouge1': 0.2541829109811566,
 'rouge2': 0.08139697175063203,
 'rougeL': 0.20759421702404157,
 'rougeLsum': 0.20759421702404157}